In [20]:
!pip install selenium webdriver-manager

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

# 1) Setup headless Chrome + wait helper
options = webdriver.ChromeOptions()
options.add_argument("--headless")
driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    options=options
)
wait = WebDriverWait(driver, 10)

# 2) Go to the tx page and wait for the first rows
driver.get("https://testnet.cintara.io/tx")
wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "tbody tr")))

# 3) Read total number of pages
total_pages = int(
    driver.find_element(
        By.CSS_SELECTOR,
        "div.pagination div.centerDetail h5.allCounting"
    ).text
)
print(f"→ Found {total_pages} pages of transactions.")

# 4) Row-parsing helper
def parse_row(tr):
    cells = tr.find_elements(By.TAG_NAME, "td")
    if len(cells) < 8:
        return None
    return {
        "Txn Hash":    cells[0].text,
        "Type":        cells[1].text,
        "Block":       cells[2].text,
        "From":        cells[3].text,
        "To":          cells[4].text,
        "Timestamp":   cells[5].text,
        "Txn Fee":     cells[6].text,
        "Value(CINT)": cells[7].text,
        "Status":      cells[8].text if len(cells) > 8 else ""
    }

# 5) Loop through pages
all_data = []
current_page = 1

while True:
    # a) scrape this page’s rows
    rows = driver.find_elements(By.CSS_SELECTOR, "tbody tr")
    for r in rows:
        rec = parse_row(r)
        if rec:
            all_data.append(rec)
    print(f"• Page {current_page}: grabbed {len(rows)} rows (total so far: {len(all_data)})")

    # b) stop if we’re on the last page
    if current_page >= total_pages:
        break

    # c) click the “>” arrow by targeting the span with aria-label="right"
    right_arrow = wait.until(EC.element_to_be_clickable((
        By.CSS_SELECTOR,
        "div.pagination div.rightArrow.false span[aria-label='right']"
    )))
    right_arrow.click()

    # d) wait until the page number increments
    next_page = current_page + 1
    wait.until(lambda d: int(
        d.find_element(
            By.CSS_SELECTOR,
            "div.pagination div.centerDetail h5.countin"
        ).text
    ) == next_page)
    current_page = next_page

# 6) Tear down and save
driver.quit()
df = pd.DataFrame(all_data)
df.to_csv("Transactions.csv", index=False)
print(f"{len(df)} transactions scraped!")



→ Found 529 pages of transactions.
• Page 1: grabbed 10 rows (total so far: 10)
• Page 2: grabbed 10 rows (total so far: 20)
• Page 3: grabbed 0 rows (total so far: 20)
• Page 4: grabbed 10 rows (total so far: 30)
• Page 5: grabbed 10 rows (total so far: 40)
• Page 6: grabbed 10 rows (total so far: 50)
• Page 7: grabbed 10 rows (total so far: 60)
• Page 8: grabbed 10 rows (total so far: 70)
• Page 9: grabbed 0 rows (total so far: 70)
• Page 10: grabbed 10 rows (total so far: 80)
• Page 11: grabbed 10 rows (total so far: 90)
• Page 12: grabbed 10 rows (total so far: 100)
• Page 13: grabbed 10 rows (total so far: 110)
• Page 14: grabbed 10 rows (total so far: 120)
• Page 15: grabbed 0 rows (total so far: 120)
• Page 16: grabbed 10 rows (total so far: 130)
• Page 17: grabbed 10 rows (total so far: 140)
• Page 18: grabbed 10 rows (total so far: 150)
• Page 19: grabbed 10 rows (total so far: 160)
• Page 20: grabbed 10 rows (total so far: 170)
• Page 21: grabbed 10 rows (total so far: 180)
•